<a href="https://colab.research.google.com/github/gaboojie/project_voting/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Plan:
1. Load CSV va_voting data - Gabe
2. Clean data into proper format for ML model - Gabe
3. Split into training and testing data - Naad
4. Choose correct ML model - Naad
5. Perform ML model - Naad
6. Calculate metrics to assess ML model - Ben
7. Create a state map based on 2024 inputs to the model - Ben

In [41]:
# Load CSV va_voting data

import pandas as pd
import numpy as np

csv_df = pd.read_csv("https://raw.githubusercontent.com/gaboojie/project_voting/main/data/voting_VA.csv")

In [42]:
# Clean data into proper format for ML model

# Independent variables to use:
# - Year
# - County Name
# - Total votes
# - Party (Republican or Democrat)

# Dependent variables to use:
# - Candidate votes

# Select the columns from the df that we will use
#df = pd.concat([csv_df['year'], csv_df['county_name'], csv_df['party'], csv_df['totalvotes'], csv_df['candidatevotes']], axis=1)

# Rename columns
df = csv_df.rename(columns={'year': 'Year', 'county_name': 'County Name', 'party': 'Party', 'totalvotes': 'Total Votes', 'candidatevotes': 'Candidate Votes', 'candidate': 'Candidate'})

# Clean Party: only include Republican and Democrat parties, drop any other party rows
df_index = df[ (df['Party'] != 'REPUBLICAN') & (df['Party'] != 'DEMOCRAT') ].index
df = df.drop(df_index)
print("Unique parties:", df['Party'].unique())

# Clean Year: Already looks good
print("Unique years:", df['Year'].unique())

# Clean County name: Already looks good
#print("Unique counties:", df['County Name'].unique())
print('Number of NAs in counties:', df['County Name'].isna().sum())

# Clean total votes: Drop any total where votes are less than 1 (there were two outliers for 2016 in Bedford with a vote count of 0)
df_index = df[df['Total Votes'] <= 0].index
df = df.drop(df_index)
print("Number of NAs in Total Votes:", df['Total Votes'].isna().sum())
#print(df['Total Votes'].unique())

# Merge 2020 absentee, election day, and provisional votes to one candidate vote
county_total_votes = df[['County Name', 'Candidate', 'Year', 'Total Votes']].drop_duplicates()

biden_entries = df[(df['Candidate'] == 'JOSEPH R BIDEN JR') & (df['Year'] == 2020)].groupby(['County Name', 'Candidate', 'Year', 'Party', 'county_fips']).agg({'Candidate Votes': 'sum'}).reset_index()
biden_entries = pd.merge(biden_entries, county_total_votes, on=['County Name', 'Candidate', 'Year'], how='left')
df = df[df['Candidate'] != 'JOSEPH R BIDEN JR']
df = pd.concat([df, biden_entries], ignore_index=True)
print(biden_entries)

trump_entries = df[(df['Candidate'] == 'DONALD J TRUMP') & (df['Year'] == 2020)].groupby(['County Name', 'Candidate', 'Year', 'Party', 'county_fips']).agg({'Candidate Votes': 'sum'}).reset_index()
trump_entries = pd.merge(trump_entries, county_total_votes, on=['County Name', 'Candidate', 'Year'], how='left')
df = df[df['Candidate'] != 'DONALD J TRUMP']
df = pd.concat([df, trump_entries], ignore_index=True)
print(trump_entries)

# Clean candidate votes: Drop any total where votes are less than 1 (there were two outliers for 2016 in Bedford with a vote count of 0)
df_index = df[df['Candidate Votes'] <= 0].index
df = df.drop(df_index)
print("Number of NAs in Candidate Votes:", df['Total Votes'].isna().sum())
#print(df['Candidate Votes'].unique())

display(df)

Unique parties: ['DEMOCRAT' 'REPUBLICAN']
Unique years: [2000 2004 2008 2012 2016 2020]
Number of NAs in counties: 0
Number of NAs in Total Votes: 0
           County Name          Candidate  Year     Party  county_fips  \
0             ACCOMACK  JOSEPH R BIDEN JR  2020  DEMOCRAT        51001   
1            ALBEMARLE  JOSEPH R BIDEN JR  2020  DEMOCRAT        51003   
2      ALEXANDRIA CITY  JOSEPH R BIDEN JR  2020  DEMOCRAT        51510   
3            ALLEGHANY  JOSEPH R BIDEN JR  2020  DEMOCRAT        51005   
4               AMELIA  JOSEPH R BIDEN JR  2020  DEMOCRAT        51007   
..                 ...                ...   ...       ...          ...   
128  WILLIAMSBURG CITY  JOSEPH R BIDEN JR  2020  DEMOCRAT        51830   
129    WINCHESTER CITY  JOSEPH R BIDEN JR  2020  DEMOCRAT        51840   
130               WISE  JOSEPH R BIDEN JR  2020  DEMOCRAT        51195   
131              WYTHE  JOSEPH R BIDEN JR  2020  DEMOCRAT        51197   
132               YORK  JOSEPH R BIDE

,Unnamed: 0,Year,state,state_po,County Name,county_fips,office,Candidate,Party,Candidate Votes,Total Votes,version,mode
0,11161.0,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,11925,20220315.0,TOTAL
1,11162.0,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315.0,TOTAL
2,11165.0,2000,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,36846,20220315.0,TOTAL
3,11166.0,2000,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,18291,36846,20220315.0,TOTAL
4,11169.0,2000,VIRGINIA,VA,ALLEGHANY,51005,US PRESIDENT,AL GORE,DEMOCRAT,2214,5123,20220315.0,TOTAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,NaN,2020,NaN,NaN,WILLIAMSBURG CITY,51830,NaN,DONALD J TRUMP,REPUBLICAN,1963,6890,NaN,NaN
1600,NaN,2020,NaN,NaN,WINCHESTER CITY,51840,NaN,DONALD J TRUMP,REPUBLICAN,5221,12113,NaN,NaN
1601,NaN,2020,NaN,NaN,WISE,51195,NaN,DONALD J TRUMP,REPUBLICAN,13366,16615,NaN,NaN
1602,NaN,2020,NaN,NaN,WYTHE,51197,NaN,DONALD J TRUMP,REPUBLICAN,11733,15073,NaN,NaN


In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

# Separate the data into training and testing sets
# Testing data is from the year 2020
train_df = df[df['Year'] != 2020]
test_df = df[df['Year'] == 2020]

# Check the distribution of training and testing data
print(f"Training data shape: {train_df.shape}")
print(f"Testing data shape: {test_df.shape}")

train_df.head(), test_df.head()

Training data shape: (1338, 13)
Testing data shape: (266, 13)


(   Unnamed: 0  Year     state state_po County Name  county_fips        office  \
 0     11161.0  2000  VIRGINIA       VA    ACCOMACK        51001  US PRESIDENT   
 1     11162.0  2000  VIRGINIA       VA    ACCOMACK        51001  US PRESIDENT   
 2     11165.0  2000  VIRGINIA       VA   ALBEMARLE        51003  US PRESIDENT   
 3     11166.0  2000  VIRGINIA       VA   ALBEMARLE        51003  US PRESIDENT   
 4     11169.0  2000  VIRGINIA       VA   ALLEGHANY        51005  US PRESIDENT   
 
         Candidate       Party  Candidate Votes  Total Votes     version   mode  
 0         AL GORE    DEMOCRAT             5092        11925  20220315.0  TOTAL  
 1  GEORGE W. BUSH  REPUBLICAN             6352        11925  20220315.0  TOTAL  
 2         AL GORE    DEMOCRAT            16255        36846  20220315.0  TOTAL  
 3  GEORGE W. BUSH  REPUBLICAN            18291        36846  20220315.0  TOTAL  
 4         AL GORE    DEMOCRAT             2214         5123  20220315.0  TOTAL  ,
       Unname

In [58]:
# Create a linear regression model using the train data
X_train = pd.concat([
    pd.get_dummies(train_df['Candidate'], dtype='int'),
    pd.get_dummies(train_df['County Name'], dtype='int'),
    pd.get_dummies(train_df['Party'], dtype='int'),
    train_df[['Total Votes']]
], axis=1)
X_train.columns = X_train.columns.astype(str)
y_train = train_df['Candidate Votes']

X = pd.get_dummies(df['Candidate'], dtype='int')  # One-hot encode the 'Candidate' column
X_party = pd.get_dummies(df['Party'], dtype='int')  # One-hot encode the 'Party' column
y = df['Candidate Votes']  # Assuming 'Candidate Votes' is your target variable

# Add the 'Total Votes' column as an additional feature
X['Total Votes'] = df['Total Votes']

# Use PolynomialFeatures to generate interaction terms up to degree 2
expander = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_interactions = expander.fit_transform(X)

# Convert X_interactions to a DataFrame
X_interactions_df = pd.DataFrame(X_interactions, columns=expander.get_feature_names_out(X.columns))

# Concatenate X_interactions_df with the party dummy variables
X_final = pd.concat([X_interactions_df, X_party], axis=1)

# Fit a linear regression model
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=False).fit(X_final, y)

results = pd.DataFrame({'Variables':reg.feature_names_in_, 'Coefficient':reg.coef_})
results





,Variables,Coefficient
0,AL GORE,1360.531534
1,BARACK OBAMA,54.038595
2,DONALD J TRUMP,2140.705850
3,DONALD TRUMP,1624.421580
4,GEORGE W. BUSH,-1132.585309
...,...,...
63,JOSEPH R BIDEN JR MITT ROMNEY,0.000000
64,JOSEPH R BIDEN JR Total Votes,0.230242
65,MITT ROMNEY Total Votes,-0.027014
66,DEMOCRAT,-1853.611649


In [49]:
X_test = pd.concat([
    pd.get_dummies(test_df['Year'], dtype='int'),
    pd.get_dummies(test_df['County Name'], dtype='int'),
    pd.get_dummies(test_df['Party'], dtype='int'),
    test_df[['Total Votes']]
], axis=1)
X_test.columns = X_test.columns.astype(str)
y_test = test_df['Candidate Votes']

# Calculate R^2
r2 = reg.score(X, y)
print('R_squared: ', r2)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


ValueError: X has 66 features, but LinearRegression is expecting 180 features as input.

In [7]:
# Calculate total votes growth rates
total_votes = df.groupby(['County Name', 'Year'])['Total Votes'].sum().reset_index()

# Merge DataFrame with itself, shifted by one term, to align total votes for consecutive
total_votes_shifted = total_votes.copy()
total_votes_shifted['Year'] = total_votes_shifted['Year'] + 4
total_votes_merged = pd.merge(total_votes, total_votes_shifted, on=['County Name', 'Year'], suffixes=('', '_prev'), how='left')

# Calculate growth rate as the percentage change between total votes in current term and previous term
total_votes_merged['Growth Rate'] = ((total_votes_merged['Total Votes'] - total_votes_merged['Total Votes_prev']) / total_votes_merged['Total Votes_prev']) * 100
print(total_votes_merged)

average_growth_rate = total_votes_merged.groupby('County Name')['Growth Rate'].mean().reset_index()

# Display the average growth rate per county
print(average_growth_rate)
average_growth_rate.head()

    County Name  Year  Total Votes  Total Votes_prev  Growth Rate
0      ACCOMACK  2000        23850               NaN          NaN
1      ACCOMACK  2004        26712           23850.0    12.000000
2      ACCOMACK  2008        31246           26712.0    16.973645
3      ACCOMACK  2012        32102           31246.0     2.739551
4      ACCOMACK  2016        31636           32102.0    -1.451623
..          ...   ...          ...               ...          ...
773        YORK  2004        59760           49166.0    21.547411
774        YORK  2008        67794           59760.0    13.443775
775        YORK  2012        67906           67794.0     0.165206
776        YORK  2016        68236           67906.0     0.485966
777        YORK  2020        77582           68236.0    13.696582

[778 rows x 5 columns]
         County Name  Growth Rate
0           ACCOMACK     7.498768
1          ALBEMARLE    12.042985
2         ALEXANDRIA     8.342775
3    ALEXANDRIA CITY          NaN
4          ALL

,County Name,Growth Rate
0,ACCOMACK,7.498768
1,ALBEMARLE,12.042985
2,ALEXANDRIA,8.342775
3,ALEXANDRIA CITY,NaN
4,ALLEGHANY,10.861802


In [8]:
# Use growth rates and 2020 total votes to derive 2024 total votes
estimated_votes_2024 = total_votes_merged[['County Name', 'Year', 'Total Votes']]
df_index = estimated_votes_2024[estimated_votes_2024['Year'] != 2020].index
estimated_votes_2024 = estimated_votes_2024.drop(df_index)
estimated_votes_2024 = pd.merge(estimated_votes_2024, average_growth_rate, on=['County Name'], how='left')
estimated_votes_2024['Estimated Total Votes 2024'] = estimated_votes_2024['Total Votes'] * (1 + average_growth_rate['Growth Rate'] / 100)
print(estimated_votes_2024)

           County Name  Year  Total Votes  Growth Rate  \
0             ACCOMACK  2020        33924     7.498768   
1            ALBEMARLE  2020       129314    12.042985   
2      ALEXANDRIA CITY  2020       165042          NaN   
3            ALLEGHANY  2020        16406    10.861802   
4               AMELIA  2020        15788    10.708869   
..                 ...   ...          ...          ...   
128  WILLIAMSBURG CITY  2020        13780          NaN   
129    WINCHESTER CITY  2020        24226          NaN   
130               WISE  2020        33230     4.615644   
131              WYTHE  2020        30146     8.195736   
132               YORK  2020        77582     9.867788   

     Estimated Total Votes 2024  
0                  36467.882026  
1                 144887.264999  
2                 178811.082656  
3                           NaN  
4                  17502.861378  
..                          ...  
128                14921.414350  
129                         NaN

In [20]:
# Use linear regression model to predict democrat and republican votes
dem_df = pd.merge(df, estimated_votes_2024[['County Name', 'Estimated Total Votes 2024']], on='County Name', how='left')
dem_df = dem_df.drop(['Total Votes'], axis=1)
dem_df = dem_df.rename(columns={'Estimated Votes 2024': 'Total Votes'})

X = pd.concat([
    pd.get_dummies(dem_df['Candidate'], dtype='int'),
    pd.get_dummies(dem_df['County Name'], dtype='int'),
    pd.get_dummies(dem_df['Party'], dtype='int'),
    df[['Total Votes']]
], axis=1)
X.columns = X.columns.astype(str)
df_index = X[X['JOSEPH R BIDEN JR'] != 1].index
reg_dem_df = X.drop(df_index)
df_index = dem_df[dem_df['Candidate'] != 'JOSEPH R BIDEN JR'].index
dem_df = dem_df.drop(df_index)

dem_df['Candidate Votes'] = reg.predict(reg_dem_df)
print(dem_df)
dem_df.head()

rep_df = pd.merge(df, estimated_votes_2024[['County Name', 'Estimated Total Votes 2024']], on='County Name', how='left')
rep_df = rep_df.drop(['Total Votes'], axis=1)
rep_df = rep_df.rename(columns={'Estimated Votes 2024': 'Total Votes'})
X = pd.concat([
    pd.get_dummies(rep_df['Candidate'], dtype='int'),
    pd.get_dummies(rep_df['County Name'], dtype='int'),
    pd.get_dummies(rep_df['Party'], dtype='int'),
    df[['Total Votes']]
], axis=1)
X.columns = X.columns.astype(str)
df_index = X[X['DONALD J TRUMP'] != 1].index
reg_rep_df = X.drop(df_index)
df_index = rep_df[rep_df['Candidate'] != 'DONALD J TRUMP'].index
rep_df = rep_df.drop(df_index)

rep_df['Candidate Votes'] = reg.predict(reg_rep_df) * -1
print(rep_df)


      Unnamed: 0  Year state state_po        County Name  county_fips office  \
1338         NaN  2020   NaN      NaN           ACCOMACK        51001    NaN   
1339         NaN  2020   NaN      NaN          ALBEMARLE        51003    NaN   
1340         NaN  2020   NaN      NaN    ALEXANDRIA CITY        51510    NaN   
1341         NaN  2020   NaN      NaN          ALLEGHANY        51005    NaN   
1342         NaN  2020   NaN      NaN             AMELIA        51007    NaN   
...          ...   ...   ...      ...                ...          ...    ...   
1466         NaN  2020   NaN      NaN  WILLIAMSBURG CITY        51830    NaN   
1467         NaN  2020   NaN      NaN    WINCHESTER CITY        51840    NaN   
1468         NaN  2020   NaN      NaN               WISE        51195    NaN   
1469         NaN  2020   NaN      NaN              WYTHE        51197    NaN   
1470         NaN  2020   NaN      NaN               YORK        51199    NaN   

              Candidate     Party  Cand

In [36]:
# Calcute net votes for each county using predicted values
df_2024 = pd.concat([dem_df, rep_df], ignore_index=True)
from google.colab import drive
drive.mount('/content/drive')
df_2024.to_csv('/content/drive/My Drive/filename.csv', index=False)
df_2024 = df_2024.groupby('County Name')['Candidate Votes'].sum().reset_index()

print(df_2024)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
           County Name  Candidate Votes
0             ACCOMACK      3392.015038
1            ALBEMARLE      3392.015038
2      ALEXANDRIA CITY      3392.015038
3            ALLEGHANY      3392.015038
4               AMELIA      3392.015038
..                 ...              ...
128  WILLIAMSBURG CITY      3392.015038
129    WINCHESTER CITY      3392.015038
130               WISE      3392.015038
131              WYTHE      3392.015038
132               YORK      3392.015038

[133 rows x 2 columns]


In [ ]:
# Graph net vote values on VA map